In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("dynamic_pricing.csv")
df.head()

In [ ]:
def category_to_columns(df: pd.DataFrame, column: str) -> pd.DataFrame:
    df_dummies = pd.get_dummies(df[column], prefix=column, dtype=int)
    df = df.drop(column, axis=1)
    return pd.concat([df, df_dummies], axis=1)

In [ ]:
df = category_to_columns(df, "Location_Category")
df = category_to_columns(df, "Customer_Loyalty_Status")
df = category_to_columns(df, "Time_of_Booking")
df = category_to_columns(df, "Vehicle_Type")
df.head()

In [ ]:
df.select_dtypes(include='number').corr(method="pearson").round(3)